## Imports

In [ ]:
# image
from skimage import io, data, filters, feature
from skimage.util import crop
import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from scipy.ndimage import distance_transform_edt
import timeit

# standard
import pandas as pd
import matplotlib.pyplot as plt

# xml parsing
from bs4 import BeautifulSoup as bs
from google.colab import drive
drive.mount('/content/drive')

# saving objects
import os
import h5py
import sys

# paths
# matthew
path_data = "/content/drive/MyDrive/NOSES/cnn_data/"
path_images = "/content/drive/MyDrive/NOSES/image_data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(os.getcwd())

/content


## Function to Parse an XML

In [ ]:
# parses xml, returns a dataframe with label and bounding box info
def parse_xml(xml):
  label = xml.find_all("name")
  xmin = xml.find_all("xmin")
  ymin = xml.find_all("ymin")
  xmax = xml.find_all("xmax")
  ymax = xml.find_all("ymax")

  for i in range(len(label)):
    label[i] = label[i].text
    xmin[i] = xmin[i].text
    ymin[i] = ymin[i].text
    xmax[i] = xmax[i].text
    ymax[i] = ymax[i].text 

  df = pd.DataFrame({"label": label, "xmin": xmin, "ymin": ymin, 
                    "xmax": xmax, "ymax": ymax})
  return df


## Read in the .JPG files

In [ ]:
start_time = timeit.default_timer()

first_fnum = 1 # first file number
last_fnum = 79 # last file number
          
imgs = []
for fnum in range(first_fnum, last_fnum + 1):
  path_to_img = path_images + "DJI_" + str(fnum).zfill(4) + ".JPG"
  img = io.imread(path_to_img, plugin="matplotlib")
  imgs.append(img)

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60)

Time (minutes) elapsed for this cell: 0.6961579448333358


## Read in the XML files

In [ ]:
from bs4 import BeautifulSoup as bs

xmls = []
for fnum in range(first_fnum, last_fnum + 1):
  content = []
  # read the XML file
  path_to_xml = path_images + "DJI_" + str(fnum).zfill(4) + ".xml"
  with open(path_to_xml, "r") as file:
      # read each line in the file, readlines() returns a list of lines
      content = file.readlines()
      # combine the lines in the list into a string
      content = "".join(content)
      xml = bs(content, "lxml")
      xmls.append(xml)


## Transfer all XML data to a Pandas DataFrame

In [ ]:
df = pd.DataFrame() # df containing all bounding boxes for all images
for fnum in range(first_fnum, last_fnum + 1):
  df_i = parse_xml(xmls[fnum - 1])
  df_i.insert(0, "file_num", str(fnum).zfill(4))
  df = pd.concat([df, df_i])

print(df.shape)
df.head()

(5848, 6)


,file_num,label,xmin,ymin,xmax,ymax
0,0001,adult male,2240,1310,2319,1379
1,0001,adult male,1344,1931,1416,1984
0,0002,adult male,3550,616,3610,677
1,0002,male juvenile / female juvenile / adult female,522,120,586,182
2,0002,adult male,216,1539,275,1654


### Clean Up Labels for Seal Categories

In [ ]:
cat1 = 'adult male'
cat2 = 'male juvenile / female juvenile / adult female'
cat3 = 'pup'

cat1_diffs = ["Adult Male", "Adult male", "adult male seal"]

cat2_diffs = ['juvenile male / juvenile female / adult female',
              'male juvenile / female juvenile /adult female',
              'Adult Female or Young Male',
              'male juvenile/female juvenile/adult female',
              'Juvenile male / juvenile female / adult female',
              'Juvenile male/ juvenile female/ adult female',
              'male juvenile / female juvenile / adulf female',
              'adult female seal', '\\\\', ']', '\'']

cat3_diffs = ['Pup', 'Juvenile', 'baby seal', 'juvenile seal']

for cat1_diff in cat1_diffs: 
  df.loc[df["label"] == cat1_diff, ['label']] = cat1

for cat2_diff in cat2_diffs: 
  df.loc[df["label"] == cat2_diff, ['label']] = cat2

for cat3_diff in cat3_diffs: 
  df.loc[df["label"] == cat3_diff, ['label']] = cat3

df['label'].unique()

array(['adult male', 'male juvenile / female juvenile / adult female',
       'pup'], dtype=object)

## Functions to split and label the images

In [ ]:
# ratio of seal present in subimage to be considered a "full" seal
PERCENT_SEAL_PRESENT = .7 

def split_image(img, num_subimgs, df_img, x_size, y_size, x_int, y_int):
    # _size is the length in each direction
    # _int is the interval you shift right or down by
    # ex. (150, 150, 75, 75) - each split creates a 150x150 images and shifts over by 75 pixels each run through.
    x_len = img.shape[1]
    y_len = img.shape[0]

    bb_data = {}
    sub_images = []
    num_image = num_subimgs
    # crops like reading a book
    i = 0
    while (i < y_len):
        # updates the new y coordinates
        y1 = i
        if i + y_size > y_len:
            y1 = y_len-y_size
            y2 = y_len
            i = y_len
        else:
            y2 = i + y_size
            i += y_int
        
        j = 0
        while (j < x_len):
            # updates the new x coordinates
            x1 = j
            if j + x_size > x_len:
                x1 = x_len - x_size
                x2 = x_len
                j = x_len
            else:
                x2 = j + x_size
                j += x_int
            df_bb = find_bounding_boxes(df_img, x_size, y_size, x1, y1, x2, y2)
            bb_data[num_image] = df_bb
            cropped = img[y1:y2,x1:x2]
            sub_images.append(cropped)
            num_image += 1

    bb_data = pd.Series(bb_data)    
    return sub_images, bb_data

# display a list of images
def display_images(sub_images, size):
  fig=plt.figure(figsize=(size, size))
  for i in range(len(sub_images)):
      sub_image = sub_images[i]
      fig.add_subplot(len(sub_images), 1, i + 1)
      plt.title(i)
      plt.imshow(sub_image)
  plt.show()


def find_bounding_boxes(df_img, x_size, y_size, xmin, ymin, xmax, ymax):
  col_names = list(df_img.columns)
  col_names.append("percent_seal")
  df_bb = pd.DataFrame()
  for row in df_img.itertuples():
    bb_xmin = int(row[3])
    bb_ymin = int(row[4])
    bb_xmax = int(row[5])
    bb_ymax = int(row[6])

    bb_xmin_ = max(bb_xmin - xmin, 0)
    bb_ymin_ = max(bb_ymin - ymin, 0)
    bb_xmax_ = min(bb_xmax - xmin, xmax - xmin)
    bb_ymax_ = min(bb_ymax - ymin, ymax - ymin)
    bb_row = [row[1], row[2], bb_xmin_, bb_ymin_, bb_xmax_, bb_ymax_]

    # if a bounding box was found
    if ((bb_xmin_ >= 0 and bb_xmin_ <= x_size) and
       (bb_ymin_ >= 0 and bb_ymin_ <= y_size) and
       (bb_xmax_ >= 0 and bb_xmax_ <= x_size) and
       (bb_ymax_ >= 0 and bb_ymax_ <= y_size)):
      height = bb_ymax_ - bb_ymin_; length = bb_xmax_ - bb_xmin_
      area = height * length # area of bb in image
      bb_area = (bb_xmax - bb_xmin) * (bb_ymax - bb_ymin) # total area of bb 

      if (bb_area == 0): # ignore bounding boxes that don't have any area
        break

      percent_seal_present = area/bb_area # % of bb present in subimage
      # print("original bb coordinates:", row)
      # print("subimage coordinates:", [xmin, ymin, xmax, ymax])
      # print("new bb coordinates:", bb_row)
      # print(percent_seal_present, area, bb_area)
      bb_row.append(percent_seal_present)
      df_bb = df_bb.append(pd.Series(bb_row, index=col_names), ignore_index=True)

  if len(df_bb) == 0:
    df_bb = None
  return df_bb

# takes in n, which splits each image into n by n images
# returns all sub images and their corresponding meta data
def split_and_label(x_size, y_size, x_int, y_int):
  num_subimgs = 0
  all_sub_imgs = []
  all_bb_data = pd.Series(dtype='object')
  for i in range(len(imgs)):
    fnum = str(i + 1).zfill(4)
    df_img = df[df["file_num"] == fnum]
    sub_imgs, bb_data = split_image(imgs[i], num_subimgs, df_img, x_size, y_size, x_int, y_int)
    all_sub_imgs += sub_imgs
    all_bb_data = pd.concat([all_bb_data, bb_data])
    num_subimgs += len(sub_imgs)
  return all_sub_imgs, all_bb_data


### Split all images into n by n subimages and label each subimage with the appropriate meta data

In [ ]:
start_time = timeit.default_timer()

sub_imgs, bb_data = split_and_label(150, 150, 75, 75)
#sub_images, bb_data = split_and_label(img, 100, 150, 60, 75)
print("Number of sub images:", len(sub_imgs))

elapsed = timeit.default_timer() - start_time
print("Time (minutes) elapsed for this cell:", elapsed/60)

Number of sub images: 125610
Time (minutes) elapsed for this cell: 5.4454755958833365


In [ ]:
len(sub_imgs)

125610

In [ ]:
len(df)

5848

## Save Images and Bounding Box Meta Data

Save entire dataset

In [ ]:
np.save(path_data + "images.npy", sub_imgs)
np.save(path_data + "bb_data.npy", bb_data)

Save small dataset for playing around with

In [ ]:
np.save(path_data + "images_small.npy", sub_imgs[:1000])
np.save(path_data + "bb_data_small.npy", bb_data[:1000])

Save slightly bigger dataset for playing around with

In [ ]:
np.save(path_data + "images_medium.npy", sub_imgs[:5000])
np.save(path_data + "bb_data_medium.npy", bb_data[:5000])

## Junk Code

In [ ]:
# complete = False; bb_found = False
#     # check for complete bounding box
#     if (bb_xmin > xmin and bb_ymin > ymin and 
#         bb_xmax < xmax and bb_ymax < ymax):
#        bb_found = True; complete = True
#        height = bb_ymax - bb_ymin; length = bb_xmax - bb_xmin
  
#     # check for partial bounding boxes
#     ## bb cut off on top margin
#     elif (bb_xmin > xmin and bb_xmax < xmax and bb_ymax < ymax):
#       bb_found = True
#     # bb cut off on bottom margin
#     elif (bb_xmin > xmin and bb_ymin > ymin and bb_xmax < xmax):
#       bb_found = True
#     # bb cut off on right margin
#     elif (bb_ymin > ymin and bb_xmax < xmax and bb_ymax < ymax):
#       bb_found = True
#     # bb cut off on left margin
#     elif (bb_xmin > xmin and bb_ymin > ymin and bb_ymax < ymax): 
#       bb_found = True
#     # bb cut off on top right corner
#     elif (bb_xmax > xmin and bb_ymax > ymin and
#           bb_xmin < xmin and bb_ymin < ymin):
#       bb_found = True
#     # bb cut off on top left corner
#     elif (bb_xmin < xmax and bb_ymax > ymin and
#           bb_xmax > xmax and bb_ymin < ymin):
#       bb_found = True
#     # bb cut off on bottom right corner
#     elif (bb_xmin < xmax and bb_ymin < ymax and
#          bb_xmax > xmax and bb_ymax > ymax):
#       bb_found = True
#     # bb cut off on bottom left corner
#     elif (bb_xmax > xmin and bb_ymin < ymax and
#           bb_xmin < xmin and bb_ymax > ymax):
#       bb_found = True